In [6]:
import chardet

with open('Chat.csv', 'rb') as f:
    result = chardet.detect(f.read())

print(result['encoding'])


Windows-1252


## Data Preprocessing  

In [62]:
import pandas as pd
import re
import nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.tokenizers import Tokenizer
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\subed\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [53]:
# import pandas as pd
df = pd.read_csv('Chat.csv', encoding='Windows-1252')



In [54]:
df.head()

,Question,Answer,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Answers(411)
0,What are the requirements for voting by absent...,Voters unable to vote in person on Election Da...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Absentee voting is available if you meet any o...
1,What is the voter registration deadline?,"Primary Election Date is June 7, 2022 (Registe...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,n person registration at the county clerk's of...
2,Where can I cast my vote?,"After registering to vote, your Voter Registra...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,What are the registration qualifications to vote?,Every U.S. citizen who possesses the following...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,How to register by mail to vote?,1. Complete a Mail-In Voter Registration Appli...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# Select columns that have a name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Remove columns that have only NaN values
df = df.dropna(axis=1, how='all')

In [56]:
df.head()

,Question,Answer,Answers(411)
0,What are the requirements for voting by absent...,Voters unable to vote in person on Election Da...,Absentee voting is available if you meet any o...
1,What is the voter registration deadline?,"Primary Election Date is June 7, 2022 (Registe...",n person registration at the county clerk's of...
2,Where can I cast my vote?,"After registering to vote, your Voter Registra...",NaN
3,What are the registration qualifications to vote?,Every U.S. citizen who possesses the following...,NaN
4,How to register by mail to vote?,1. Complete a Mail-In Voter Registration Appli...,NaN


In [57]:
def preprocess_text(text):
    # Loading spaCy model
    nlp = spacy.load('en_core_web_sm')

    # Check if text is a string
    if isinstance(text, str):
        # Converting text to lowercase
        text = text.lower()

        # Removing extra whitespaces
        text = ' '.join(text.split())

        # Removing punctuation
        text = re.sub(r'[^\w\s]', '', text)

        # Lemmatization
        doc = nlp(text)
        text = ' '.join([token.lemma_ for token in doc])

    return text


In [58]:
# Preprocessing text data
df['preprocessed_text'] = df['Answer'].apply(preprocess_text)


In [59]:
df['preprocessed_text'] = df['preprocessed_text'].astype(str)


In [60]:
# Saving preprocessed data as a new CSV file
df.to_csv('preprocessed_file.csv', index=False)

## NLP Analysis

In [45]:
!pip install gensim


In [46]:
from gensim.summarization import summarize

ModuleNotFoundError: No module named 'gensim.summarization'

In [75]:
# Load the preprocessed data into a string
text = ' '.join(df['preprocessed_text'].tolist())

# Set the summarizer algorithm
summarizer = LsaSummarizer()

# Set the number of sentences to include in the summary
num_sentences = 1

# Parse the text document
parser = PlaintextParser.from_string(text, Tokenizer('english'))

# Generate the summary
summary = summarizer(parser.document, num_sentences)

# Print the summary
print(summary[0])


voter unable to vote in person on election day may be eligible to vote by absentee ballot most absentee voter must appear before the circuit clerk or municipal clerk and absentee vote in person a few category of absentee voter may request a mail ballot voter who be require to be at work while the polling place be open on election day or will be out of town must absentee vote in person absentee voter who be 65 or old have a permanent or temporary physical disability or be temporarily reside outside their county of residence may absentee vote by mail please check with your circuit or municipal clerk to determine if you be entitle to vote by an absentee ballot and to learn the procedure for do so if you know you will vote by absentee ballot you may visit or contact your circuit or municipal clerkâs office within 45 day of the election voter include within the uniform and overseas citizen absentee voting act uocava such as member of the military and overseas citizen may register to vote an

In [76]:
# Initialize summarizer
summarizer = LsaSummarizer()

# Initialize tokenizer
tokenizer = Tokenizer("english")

# Loop over each row in the dataframe and summarize the text
for index, row in df.iterrows():
    text = row['preprocessed_text']
    parser = PlaintextParser.from_string(text, tokenizer)
    summary = summarizer(parser.document, 1) # 3 is the number of sentences to summarize to
    df.at[index, 'summary'] = ' '.join([str(sentence) for sentence in summary])

In [77]:
df.head()

,Question,Answer,Answers(411),preprocessed_text,summary
0,What are the requirements for voting by absent...,Voters unable to vote in person on Election Da...,Absentee voting is available if you meet any o...,voter unable to vote in person on election day...,voter unable to vote in person on election day...
1,What is the voter registration deadline?,"Primary Election Date is June 7, 2022 (Registe...",n person registration at the county clerk's of...,primary election date be june 7 2022 register ...,primary election date be june 7 2022 register ...
2,Where can I cast my vote?,"After registering to vote, your Voter Registra...",NaN,after register to vote your voter registration...,after register to vote your voter registration...
3,What are the registration qualifications to vote?,Every U.S. citizen who possesses the following...,NaN,every us citizen who possess the follow qualif...,every us citizen who possess the follow qualif...
4,How to register by mail to vote?,1. Complete a Mail-In Voter Registration Appli...,NaN,1 complete a mailin voter registration applica...,1 complete a mailin voter registration applica...


In [72]:
df['summary'][0]

'voter unable to vote in person on election day may be eligible to vote by absentee ballot most absentee voter must appear before the circuit clerk or municipal clerk and absentee vote in person a few category of absentee voter may request a mail ballot voter who be require to be at work while the polling place be open on election day or will be out of town must absentee vote in person absentee voter who be 65 or old have a permanent or temporary physical disability or be temporarily reside outside their county of residence may absentee vote by mail please check with your circuit or municipal clerk to determine if you be entitle to vote by an absentee ballot and to learn the procedure for do so if you know you will vote by absentee ballot you may visit or contact your circuit or municipal clerkâs office within 45 day of the election voter include within the uniform and overseas citizen absentee voting act uocava such as member of the military and overseas citizen may register to vote a

In [73]:
df['preprocessed_text'][0]

'voter unable to vote in person on election day may be eligible to vote by absentee ballot most absentee voter must appear before the circuit clerk or municipal clerk and absentee vote in person a few category of absentee voter may request a mail ballot voter who be require to be at work while the polling place be open on election day or will be out of town must absentee vote in person absentee voter who be 65 or old have a permanent or temporary physical disability or be temporarily reside outside their county of residence may absentee vote by mail please check with your circuit or municipal clerk to determine if you be entitle to vote by an absentee ballot and to learn the procedure for do so if you know you will vote by absentee ballot you may visit or contact your circuit or municipal clerkâs office within 45 day of the election voter include within the uniform and overseas citizen absentee voting act uocava such as member of the military and overseas citizen may register to vote a

In [74]:
# Iterate over each row
for index, row in df.iterrows():
    # Check if the preprocessed_text and summary columns are identical
    if row['preprocessed_text'] == row['summary']:
        print(f"Row {index}: Identical")
    else:
        print(f"Row {index}: Different")


Row 0: Identical
Row 1: Identical
Row 2: Identical
Row 3: Identical
Row 4: Identical
Row 5: Identical
Row 6: Identical
Row 7: Identical
Row 8: Identical
Row 9: Identical
Row 10: Identical
Row 11: Identical
Row 12: Identical
Row 13: Identical
Row 14: Identical
Row 15: Identical
Row 16: Identical
Row 17: Identical
Row 18: Identical
Row 19: Identical
Row 20: Identical
Row 21: Identical
Row 22: Identical
Row 23: Identical
Row 24: Identical
Row 25: Identical
Row 26: Identical
Row 27: Identical
Row 28: Identical
